In [ ]:
#hide
#default_exp utils.organize_experiments
from nbdev.showdoc import show_doc
import warnings
warnings.filterwarnings('ignore')

# Organize experiments

> Routines for organizing the experiments folders

In [ ]:
#export
import pandas as pd
import numpy as np
import shutil

from hpsearch.utils import experiment_utils

def join_experiments (path_source, path_destination, key_score=None):
    experiment_data_source = pd.read_pickle ('%s/experiments_data.pk' %path_source)
    experiment_data_destination = pd.read_pickle ('%s/experiments_data.pk' %path_destination)
    experiment_data_source, changed_source = remove_defaults_from_experiment_data (experiment_data_source)
    experiment_data_destination, changed_destination = remove_defaults_from_experiment_data (experiment_data_destination)
    
    for experiment_number_source in range(experiment_data_source.shape[0]):
        path_experiment_source = '%s/experiments/%05d' %(path_source, experiment_number_source)
        parameters_source, _ = pickle.load(open('%s/parameters.pk' %path_experiment_source,'rb'))
        experiment_number_destination, changed_dataframe, _ = experiment_utils.find_rows_with_parameters_dict (experiment_data_destination, parameters_source)
        path_experiment_destination = '%s/experiments/%05d' %(path_destination, experiment_number_destination)
        if changed_dataframe:
            # move folders
            os.rename (path_experiment_source, path_experiment_destination)
            # copy results to dataframe
            missing_cols = [col for col in experiment_data_source.columns if col not in experiment_data_destination.columns]
            for column in missing_cols:
                experiment_data_destination[column] = None
            experiment_data_destination.loc[experiment_number_destination] = experiment_data_source.loc[experiment_number_source]
        else:
            class_ids_source = [int(x) for x in os.listdir(path_experiment_source) if os.path.isdir('%s/%s' %(path_experiment_source, x))]
            class_ids_destination = [int(x) for x in os.listdir(path_experiment_destination) if os.path.isdir('%s/%s' %(path_experiment_destination, x))]
            last_id_destination = max(class_ids_destination)
            
            class_ids_both = [x for x in class_ids_source if x in class_ids_destination]
            class_ids_source = [x for x in class_ids_source if x not in class_ids_both]
            class_ids_destination = [x for x in class_ids_destination if x not in class_ids_both]
            for (idx, class_id_source) in enumerate(class_ids_both):
                if key_score is not None:
                    scores_name = '%d_%s' %(class_id_source, key_score)
                    if experiment_data_source.loc[experiment_number_source, scores_name] != experiment_data_destination.loc[experiment_number_destination, scores_name]:
                        is_new = True
                else:
                    is_new = False
                    scores_name_source = [x for x in experiment_data_source.columns if x.startswith('%d_' %class_id_source)]
                    scores_name_source = [x for x in scores_name_source if not np.isnan(experiment_data_source.loc[experiment_number_source, x])]
                    for scores_name in scores_name_source:
                        if experiment_data_source.loc[experiment_number_source, scores_name] != experiment_data_destination.loc[experiment_number_destination, scores_name]:
                            is_new = True
                            break
                if not is_new:
                    del class_ids_both[idx]
            class_ids_source += class_ids_both
            class_ids_destination += class_ids_both
                
            last_id_source = len(class_ids_source)
            new_ids_destination = range(last_id_destination+1, last_id_destination+last_id_source)
            for (new_id_destination, class_id_source) in zip(new_ids_destination, class_ids_source):
                # move folders
                os.rename ('%s/%d' %(path_experiment_source, class_id_source), '%s/%d' %(path_experiment_destination, new_id_destination))
                # copy results to dataframe
                scores_name_source = [x for x in experiment_data_source.columns if x.startswith('%d_' %class_id_source)]
                scores_name_destination = ['%d_%s' (new_id_destination, x[len('%d_' %class_id_source):]) for x in scores_name_source]
                for score_name_source, score_name_destination in zip(scores_name_source, scores_name_destination):
                    experiment_data_destination.loc[experiment_number_destination, score_name_destination] = experiment_data_source.loc[experiment_number_source, score_name_source]
        
        experiment_data_destination.to_csv ('%s/experiments_data.csv' %path_destination)
        experiment_data_destination.to_pickle ('%s/experiments_data.pk' %path_destination)
        
def remove_defaults_from_experiment_data (experiment_data):
    from hpsearch.config.hpconfig import get_default_parameters
    
    experiment_data_original = experiment_data.copy()
    parameters_names = experiment_utils.get_parameters_columns (experiment_data)
    parameters_data = experiment_data_original[parameters_names]
    changed_df = False
    for experiment_number in range(experiment_data.shape[0]):
        good_params = ~(experiment_data.loc[experiment_number, parameters_names].isna()).values
        parameters_names_i = np.array(parameters_names)[good_params]
        parameters_names_i = parameters_names_i.tolist()
        parameters = experiment_data.loc[experiment_number, parameters_names_i].to_dict()

        defaults = get_default_parameters(parameters)
        default_names = [default_name for default_name in defaults.keys() if default_name in parameters_names_i]
        
        for default_name in default_names:
            has_default = experiment_data.loc[experiment_number, default_name] == defaults[default_name]
            if has_default:
                print ('found experiment with default in experiment_number {}, parameter {}, values: {}'.format(experiment_number, default_name, experiment_data.loc[experiment_number, default_name]))
                changed_df = True
                experiment_data.loc[experiment_number, default_name] = None
    
    # check if there are repeated experiments
    if changed_df:
        if experiment_data[parameters_names].duplicated().any():
            print ('duplicated experiments: {}'.format(experiment_data[parameters_names].duplicated()))
            experiment_data = experiment_data_original
            changed_df = False
        
    return experiment_data, changed_df

def remove_experiments (experiments=[], root_path=None, root_folder=None):
    from hpsearch.config.hpconfig import get_path_experiment, get_path_experiments
    
    if type(experiments) is not list:
        experiments = [experiments]
    if root_path is None:
        root_path = get_path_experiments(folder = root_folder)
    
    # 1. remove experiments from csv file
    path_csv = f'{root_path}/experiments_data.csv'
    path_pickle = path_csv.replace('csv', 'pk')
    experiment_data = pd.read_pickle (path_pickle)
    experiment_data = experiment_data.drop (index=experiments)
    
    # 2. remove experiments folders
    for experiment in experiments:
        path_experiment = get_path_experiment (experiment, root_path=root_path, root_folder=root_folder)
        shutil.rmtree(path_experiment)
        
    # 3. move experiment folders
    for new_number, original_number in enumerate(experiment_data.index):
        path_new_experiment = get_path_experiment (new_number, root_path=root_path, root_folder=root_folder)
        path_original_experiment = get_path_experiment (original_number, root_path=root_path, root_folder=root_folder)
        if path_new_experiment != path_original_experiment:
            shutil.move (path_original_experiment, path_new_experiment)
            
    # 4. move experiment indexes
    experiment_data.index = range(len(experiment_data.index))
    
    # 5. save experiment data
    experiment_data.to_csv (path_csv)
    experiment_data.to_pickle (path_pickle)

## Usage example

### Produce data

In [ ]:
#collapse_output
from hpsearch.examples.dummy_experiment_manager import remove_previous_experiments, run_multiple_experiments

remove_previous_experiments()
run_multiple_experiments(nruns=5, noise=0.1)

experiment script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line: 201
processing hyper-parameter 0 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 0
run number: 0

parameters:
	epochs=5,
	noise=0.1,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0009300708770751953
0 - validation_accuracy: 0.39818992477057
0 - test_accuracy: 0.5030049616919423
finished experiment 0
processing hyper-parameter 0 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 0
run number: 1

parameters:
	epochs=5,
	noise=0.1,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0009531974792480469
1 - validation_accuracy: 0.2375319443046206
1 - test_accuracy: 0.36513440595343305
finished experiment 0
processing hyper-paramet

current path: /mnt/athena/hpsearch/nbs/utils
current path: /mnt/athena/hpsearch/nbs/utils
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
current path: /mnt/athena/hpsearch/nbs/utils


script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0009417533874511719
4 - validation_accuracy: 0.12371940103658746
4 - test_accuracy: 0.3411245339321954
finished experiment 0
processing hyper-parameter 1 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 0

parameters:
	epochs=5,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0008573532104492188
0 - validation_accuracy: 0.3792321065908727
0 - test_accuracy: 0.6546671705439889
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 1

parameters:
	epochs=5,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experim

fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
current path: /mnt/athena/hpsearch/nbs/utils
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
ep

running experiment 1
run number: 4

parameters:
	epochs=5,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0009436607360839844
4 - validation_accuracy: 0.4785422692285996
4 - test_accuracy: 0.5414592350384044
finished experiment 1
processing hyper-parameter 2 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 2
run number: 0

parameters:
	epochs=5,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0008647441864013672
0 - validation_accuracy: 0.7177950492870028
0 - test_accuracy: 0.8167067183795961
finished experiment 2
processing hyper-parameter 2 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 2
run number: 1

parameters:
	epochs=5,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpse

...

fitting model with 30 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
epoch 5: accuracy: 0.4800000000000001
epoch 6: accuracy: 0.5100000000000001
epoch 7: accuracy: 0.5400000000000001
epoch 8: accuracy: 0.5700000000000002
epoch 9: accuracy: 0.6000000000000002
epoch 10: accuracy: 0.6300000000000002
epoch 11: accuracy: 0.6600000000000003
epoch 12: accuracy: 0.6900000000000003
epoch 13: accuracy: 0.7200000000000003
epoch 14: accuracy: 0.7500000000000003
epoch 15: accuracy: 0.7800000000000004
epoch 16: accuracy: 0.8100000000000004
epoch 17: accuracy: 0.8400000000000004
epoch 18: accuracy: 0.8700000000000004
epoch 19: accuracy: 0.9000000000000005
epoch 20: accuracy: 0.8700000000000004
epoch 21: accuracy: 0.8400000000000004
epoch 22: accuracy: 0.8100000000000004
epoch 23: accuracy: 0.7800000000000004
epoch 24: accuracy: 0.7500000000000003
epoch 25: accuracy: 0.720000000

running experiment 7
run number: 4

parameters:
	epochs=30,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.003377199172973633
4 - validation_accuracy: 0.7581533238280647
4 - test_accuracy: 0.460380786978612
finished experiment 7
processing hyper-parameter 8 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 0

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.003300189971923828
0 - validation_accuracy: 0.9462807447804227
0 - test_accuracy: 0.8023390144087101
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 1

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpse

fitting model with 30 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
epoch 5: accuracy: 0.4800000000000001
epoch 6: accuracy: 0.5100000000000001
epoch 7: accuracy: 0.5400000000000001
epoch 8: accuracy: 0.5700000000000002
epoch 9: accuracy: 0.6000000000000002
epoch 10: accuracy: 0.6300000000000002
epoch 11: accuracy: 0.6600000000000003
epoch 12: accuracy: 0.6900000000000003
epoch 13: accuracy: 0.7200000000000003
epoch 14: accuracy: 0.7500000000000003
epoch 15: accuracy: 0.7800000000000004
epoch 16: accuracy: 0.8100000000000004
epoch 17: accuracy: 0.8400000000000004
epoch 18: accuracy: 0.8700000000000004
epoch 19: accuracy: 0.9000000000000005
epoch 20: accuracy: 0.8700000000000004
epoch 21: accuracy: 0.8400000000000004
epoch 22: accuracy: 0.8100000000000004
epoch 23: accuracy: 0.7800000000000004
epoch 24: accuracy: 0.7500000000000003
epoch 25: accuracy: 0.720000000

running experiment 8
run number: 3

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.0033392906188964844
3 - validation_accuracy: 0.8484903385387791
3 - test_accuracy: 0.7103962847313978
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 4

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 201
time spent on this experiment: 0.003292083740234375
4 - validation_accuracy: 0.7977704861745482
4 - test_accuracy: 0.9997575149738822
finished experiment 8


fitting model with 30 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy: 0.8700000000000002
epoch 9: accuracy: 0.9000000000000002
epoch 10: accuracy: 0.9300000000000003
epoch 11: accuracy: 0.9600000000000003
epoch 12: accuracy: 0.9900000000000003
epoch 13: accuracy: 1.0200000000000002
epoch 14: accuracy: 1.0500000000000003
epoch 15: accuracy: 1.0800000000000003
epoch 16: accuracy: 1.1100000000000003
epoch 17: accuracy: 1.1400000000000003
epoch 18: accuracy: 1.1700000000000004
epoch 19: accuracy: 1.2000000000000004
epoch 20: accuracy: 1.1700000000000004
epoch 21: accuracy: 1.1400000000000003
epoch 22: accuracy: 1.1100000000000003
epoch 23: accuracy: 1.0800000000000003
epoch 24: accuracy: 1.0500000000000003
epoch 25: accuracy: 1.020000000000

In [ ]:
from hpsearch.config.hpconfig import get_path_experiments
import os
import pandas as pd

path_experiments = get_path_experiments ()
print (f'path_experiments: {path_experiments}\n')
print (f'experiments content: {os.listdir(path_experiments)}\n')
print (f'experiments inside: {os.listdir(f"{path_experiments}/experiments")}\n')

experiments_data = pd.read_pickle (f'{path_experiments}/experiments_data.pk')
old_experiments_data = experiments_data
print (f'csv data index {experiments_data.index}\n')
print ('csv content:')
experiments_data

path_experiments: /mnt/athena/hpsearch/hpsearch/../results

experiments content: ['experiments_data.pk', 'current_experiment_number.pkl', 'summary.txt', 'experiments', 'git_hash.json', 'experiments_data.csv', 'logs.txt', 'parameters.pk', 'other_parameters.csv', 'parameters.txt']

experiments inside: ['00000', '00008', '00005', '00006', '00002', '00004', '00001', '00007', '00003']

csv data index RangeIndex(start=0, stop=9, step=1)

csv content:


,epochs,noise,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,1_validation_accuracy,...,time_2,2_finished,3_validation_accuracy,3_test_accuracy,time_3,3_finished,4_validation_accuracy,4_test_accuracy,time_4,4_finished
0,5.0,0.1,0.1,0.03,0.398190,0.503005,0.001472,15:43:50.843200,True,0.237532,...,0.001378,True,0.288024,0.324241,0.001403,True,0.123719,0.341125,0.001447,True
1,5.0,0.1,0.3,0.03,0.379232,0.654667,0.001369,15:43:51.064255,True,0.745192,...,0.001507,True,0.426478,0.558034,0.001398,True,0.478542,0.541459,0.001453,True
2,5.0,0.1,0.6,0.03,0.717795,0.816707,0.001388,15:43:51.283737,True,0.697048,...,0.001457,True,0.597638,0.839794,0.001464,True,0.835126,0.664219,0.001474,True
3,15.0,0.1,0.1,0.03,0.795840,0.383417,0.002422,15:43:51.515454,True,0.684275,...,0.002560,True,0.397632,0.402169,0.002899,True,0.552610,0.457519,0.002405,True
4,15.0,0.1,0.3,0.03,0.738689,0.709655,0.002459,15:43:51.759312,True,0.632554,...,0.002354,True,0.854771,0.830298,0.002466,True,0.661954,0.550794,0.002438,True
5,15.0,0.1,0.6,0.03,0.896727,0.912238,0.002409,15:43:52.010218,True,1.000000,...,0.002463,True,1.000000,1.000000,0.002443,True,0.949093,1.000000,0.002416,True
6,30.0,0.1,0.1,0.03,0.268727,0.190381,0.004025,15:43:52.275762,True,0.191534,...,0.003689,True,0.458991,0.281427,0.003723,True,0.379079,0.431044,0.003976,True
7,30.0,0.1,0.3,0.03,0.658142,0.533260,0.003903,15:43:52.560892,True,0.790248,...,0.003795,True,0.669601,0.542452,0.003951,True,0.758153,0.460381,0.004018,True
8,30.0,0.1,0.6,0.03,0.946281,0.802339,0.003829,15:43:52.846902,True,0.899529,...,0.004088,True,0.848490,0.710396,0.003877,True,0.797770,0.999758,0.003820,True


### Removing experiments

In [ ]:
remove_experiments (experiments=[3,7])

In [ ]:
old_experiments_data

,epochs,noise,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,1_validation_accuracy,...,time_2,2_finished,3_validation_accuracy,3_test_accuracy,time_3,3_finished,4_validation_accuracy,4_test_accuracy,time_4,4_finished
0,5.0,0.1,0.1,0.03,0.398190,0.503005,0.001472,15:43:50.843200,True,0.237532,...,0.001378,True,0.288024,0.324241,0.001403,True,0.123719,0.341125,0.001447,True
1,5.0,0.1,0.3,0.03,0.379232,0.654667,0.001369,15:43:51.064255,True,0.745192,...,0.001507,True,0.426478,0.558034,0.001398,True,0.478542,0.541459,0.001453,True
2,5.0,0.1,0.6,0.03,0.717795,0.816707,0.001388,15:43:51.283737,True,0.697048,...,0.001457,True,0.597638,0.839794,0.001464,True,0.835126,0.664219,0.001474,True
3,15.0,0.1,0.1,0.03,0.795840,0.383417,0.002422,15:43:51.515454,True,0.684275,...,0.002560,True,0.397632,0.402169,0.002899,True,0.552610,0.457519,0.002405,True
4,15.0,0.1,0.3,0.03,0.738689,0.709655,0.002459,15:43:51.759312,True,0.632554,...,0.002354,True,0.854771,0.830298,0.002466,True,0.661954,0.550794,0.002438,True
5,15.0,0.1,0.6,0.03,0.896727,0.912238,0.002409,15:43:52.010218,True,1.000000,...,0.002463,True,1.000000,1.000000,0.002443,True,0.949093,1.000000,0.002416,True
6,30.0,0.1,0.1,0.03,0.268727,0.190381,0.004025,15:43:52.275762,True,0.191534,...,0.003689,True,0.458991,0.281427,0.003723,True,0.379079,0.431044,0.003976,True
7,30.0,0.1,0.3,0.03,0.658142,0.533260,0.003903,15:43:52.560892,True,0.790248,...,0.003795,True,0.669601,0.542452,0.003951,True,0.758153,0.460381,0.004018,True
8,30.0,0.1,0.6,0.03,0.946281,0.802339,0.003829,15:43:52.846902,True,0.899529,...,0.004088,True,0.848490,0.710396,0.003877,True,0.797770,0.999758,0.003820,True


In [ ]:
experiments_data

,epochs,noise,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,1_validation_accuracy,...,time_2,2_finished,3_validation_accuracy,3_test_accuracy,time_3,3_finished,4_validation_accuracy,4_test_accuracy,time_4,4_finished
0,5.0,0.1,0.1,0.03,0.398190,0.503005,0.001472,15:43:50.843200,True,0.237532,...,0.001378,True,0.288024,0.324241,0.001403,True,0.123719,0.341125,0.001447,True
1,5.0,0.1,0.3,0.03,0.379232,0.654667,0.001369,15:43:51.064255,True,0.745192,...,0.001507,True,0.426478,0.558034,0.001398,True,0.478542,0.541459,0.001453,True
2,5.0,0.1,0.6,0.03,0.717795,0.816707,0.001388,15:43:51.283737,True,0.697048,...,0.001457,True,0.597638,0.839794,0.001464,True,0.835126,0.664219,0.001474,True
3,15.0,0.1,0.3,0.03,0.738689,0.709655,0.002459,15:43:51.759312,True,0.632554,...,0.002354,True,0.854771,0.830298,0.002466,True,0.661954,0.550794,0.002438,True
4,15.0,0.1,0.6,0.03,0.896727,0.912238,0.002409,15:43:52.010218,True,1.000000,...,0.002463,True,1.000000,1.000000,0.002443,True,0.949093,1.000000,0.002416,True
5,30.0,0.1,0.1,0.03,0.268727,0.190381,0.004025,15:43:52.275762,True,0.191534,...,0.003689,True,0.458991,0.281427,0.003723,True,0.379079,0.431044,0.003976,True
6,30.0,0.1,0.6,0.03,0.946281,0.802339,0.003829,15:43:52.846902,True,0.899529,...,0.004088,True,0.848490,0.710396,0.003877,True,0.797770,0.999758,0.003820,True


In [ ]:
(experiments_data.loc[3] == old_experiments_data.loc[4]).all()

True

In [ ]:
# we check that the remaining experiments do not contain number 3 or 7
experiment_folders = os.listdir(f"{path_experiments}/experiments")
print (f'experiment folders after removal: {experiment_folders}\n')
assert len(experiment_folders)==7 and ('00007' not in experiment_folders) and ('00008' not in experiment_folders)

# we check that neither 3 nor 7 are in the new dataframe index
experiments_data = pd.read_pickle (f'{path_experiments}/experiments_data.pk')
print (f'csv data index after removal: {experiments_data.index}\n')
assert (experiments_data.index==range(7)).all()

assert (experiments_data.loc[3] == old_experiments_data.loc[4]).all() and (experiments_data.loc[6] == old_experiments_data.loc[8]).all()
assert (experiments_data.loc[4] == old_experiments_data.loc[5]).all()

print ('csv content:')
experiments_data

experiment folders after removal: ['00000', '00005', '00006', '00002', '00004', '00001', '00003']

csv data index after removal: RangeIndex(start=0, stop=7, step=1)

csv content:


,epochs,noise,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,1_validation_accuracy,...,time_2,2_finished,3_validation_accuracy,3_test_accuracy,time_3,3_finished,4_validation_accuracy,4_test_accuracy,time_4,4_finished
0,5.0,0.1,0.1,0.03,0.398190,0.503005,0.001472,15:43:50.843200,True,0.237532,...,0.001378,True,0.288024,0.324241,0.001403,True,0.123719,0.341125,0.001447,True
1,5.0,0.1,0.3,0.03,0.379232,0.654667,0.001369,15:43:51.064255,True,0.745192,...,0.001507,True,0.426478,0.558034,0.001398,True,0.478542,0.541459,0.001453,True
2,5.0,0.1,0.6,0.03,0.717795,0.816707,0.001388,15:43:51.283737,True,0.697048,...,0.001457,True,0.597638,0.839794,0.001464,True,0.835126,0.664219,0.001474,True
3,15.0,0.1,0.3,0.03,0.738689,0.709655,0.002459,15:43:51.759312,True,0.632554,...,0.002354,True,0.854771,0.830298,0.002466,True,0.661954,0.550794,0.002438,True
4,15.0,0.1,0.6,0.03,0.896727,0.912238,0.002409,15:43:52.010218,True,1.000000,...,0.002463,True,1.000000,1.000000,0.002443,True,0.949093,1.000000,0.002416,True
5,30.0,0.1,0.1,0.03,0.268727,0.190381,0.004025,15:43:52.275762,True,0.191534,...,0.003689,True,0.458991,0.281427,0.003723,True,0.379079,0.431044,0.003976,True
6,30.0,0.1,0.6,0.03,0.946281,0.802339,0.003829,15:43:52.846902,True,0.899529,...,0.004088,True,0.848490,0.710396,0.003877,True,0.797770,0.999758,0.003820,True


In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)